In [4]:
print("\u03B5")
print("ε")

EPSILON = "\u03B5"

ε
ε


In [ ]:
class Edge:
    def __init__(self, source = None, destination = None, label = None):
        self.source = source
        self.destination = destination
        self.label = label
        
    def __repr__(self):
        src_label = "None" if self.source is None else "State"
        dest_label = "None" if self.destination is None else "State"
        return f"Edge({src_label} --[{self.label}]--> {dest_label})"

In [ ]:
class State:
    def __init__(self, id = None):
        self.id = id
        self.outgoing_edges = []
        
    def add_edge(self, destination, label):
        new_edge = Edge(self, destination, label)
        self.outgoing_edges.append(new_edge)
        
    def __repr__(self):
        return f"State({self.id})"

In [ ]:
class NFA:
    def __init__(self, initial_state, inner_states, accepting_states ):
        self.initial_state = initial_state
        self.accepting_states = accepting_states
        self.inner_states = inner_states

In [ ]:
def zero_or_more(stack, id):
    # Only pop 1 NFA from the stack
    nfa = stack.pop()
    
    ## Create 2 new states
    new_start = State("S" + str(id))
    new_end = State("S" + str(id + 1))
    
    """
    Create 4 new edges:
    - one from new start state to old start state
    - one from new start state to new end state
    - one from old end state to new start state
    - one from old end state to new end state
    """
    new_start.add_edge(nfa.initial_state, EPSILON)
    new_start.add_edge(new_end, EPSILON)

    for acc_state in nfa.accepting_states:
        acc_state.add_edge(nfa.initial_state, EPSILON)
        acc_state.add_edge(new_end, EPSILON)

    result_nfa = NFA(
        initial_state=new_start,
        inner_states=[new_start, new_end] + nfa.inner_states,
        accepting_states=[new_end] 
    )
    
    return result_nfa, id + 2

In [ ]:
def one_or_more(stack, counter_id):
    # Only pop 1 NFA from the stack
    nfa = stack.pop()
    
    ## Create 2 new states
    new_start = State("S" + str(counter_id))
    new_end = State("S" + str(counter_id + 1))
    
    """
    Create 4 new edges:
    - one from new start state to old start state
    - one from old end state to new start state
    - one from old end state to new end state
    """
    new_start.add_edge(nfa.initial_state, EPSILON)

    for acc_state in nfa.accepting_states:
        acc_state.add_edge(nfa.initial_state, EPSILON)
        acc_state.add_edge(new_end, EPSILON)

    result_nfa = NFA(
        initial_state=new_start,
        inner_states=[new_start, new_end] + nfa.inner_states,
        accepting_states=[new_end] 
    )
    
    return result_nfa, counter_id + 2

In [ ]:
def zero_or_one(stack, counter_id):
    # Only pop 1 NFA from the stack
    nfa = stack.pop()
    
    ## Create 2 new states
    new_start = State("S" + str(counter_id))
    new_end = State("S" + str(counter_id + 1))
    
    """
    Create 3 new edges:
    - one from new start state to old start state
    - one from new start state to new end state
    - one from old end state to new end state
    """
    new_start.add_edge(nfa.initial_state, EPSILON)
    new_start.add_edge(new_end, EPSILON)

    for acc_state in nfa.accepting_states:
        acc_state.add_edge(new_end, EPSILON)

    result_nfa = NFA(
        initial_state=new_start,
        inner_states=[new_start, new_end] + nfa.inner_states,
        accepting_states=[new_end] 
    )
    
    return result_nfa, counter_id + 2

In [ ]:
def concat(stack):
    # Pop 2 NFAs from the stack
    end_nfa = stack.pop()
    start_nfa = stack.pop()
    
    ## Add an edge from the accepting states of the first NFA to the second one
    for acc_state in start_nfa.accepting_states:
        acc_state.add_edge(end_nfa.initial_state, EPSILON)
             
    result_nfa = NFA(
        initial_state=start_nfa.initial_state,
        inner_states=start_nfa.inner_states + end_nfa.inner_states,
        accepting_states= end_nfa.accepting_states
    )
    
    return result_nfa

In [ ]:
def oring(stack, counter_id):
    ## Create 2 new states
    new_start = State("S" + str(counter_id))
    new_end = State("S" + str(counter_id + 1))
    
    ## Pop 2 NFAs from the stack
    first_nfa = stack.pop()
    second_nfa = stack.pop()
    
    ## Add 2 edges (from the new start state to the old start ones)
    new_start.add_edge(first_nfa.initial_state, EPSILON)
    new_start.add_edge(second_nfa.initial_state, EPSILON)
    
    ## Add an edge for each of the old ending states to the new ones
    for acc_state in first_nfa.accepting_states:
        acc_state.add_edge(new_end, EPSILON)
        
    for acc_state in second_nfa.accepting_states:
        acc_state.add_edge(new_end, EPSILON)
        
    result_nfa = NFA(
        initial_state=new_start,
        inner_states=[new_start, new_end] + first_nfa.inner_states + second_nfa.inner_states,
        accepting_states=[new_end] 
    )
    
    return result_nfa, counter_id + 2

In [ ]:
def create_nfa(c, counter_id):
    ## Create 2 new states
    new_start = State("S" + str(counter_id))
    new_end = State("S" + str(counter_id + 1))
    
    # Create an edge connecting both states with the character as a label
    new_start.add_edge(new_end, c)
    
    new_nfa = NFA(
        initial_state=new_start,
        inner_states=[new_start, new_end],
        accepting_states=[new_end]
    )
    
    return new_nfa, counter_id + 2

In [ ]:
def construct_nfa(postfix):
    stack = []
    counter = 0
    
    for char in postfix:
        if char == "&":
            new_nfa = concat(stack)
        elif char == "|":
            new_nfa, counter = oring(stack, counter)
        elif char == '+':
            new_nfa, counter = one_or_more(stack, counter)
        elif char =='*':
            new_nfa, counter = zero_or_more(stack, counter)
        elif char == '?':
            new_nfa, counter = zero_or_one(stack, counter)
        else:
            new_nfa, counter = create_nfa(char, counter)
        
        stack.append(new_nfa)
        
        if stack:
            return stack.pop()
        else:
            print("An error occurred while creating NFA")
            return False